## Tests crashs

### Implémentation de la gestion des crashs par exclusion

Si un point fait crasher le logiciel, on le rééssaie 3 fois et si il ne veut vraiment pas, on renvoie à l'algorithme l'infini pour sortir de la zone problématique.

In [ ]:
import openturns as ot
import numpy as np
import subprocess
import time

class MyModel2:
    def __init__(self, noms_params, Re, penalty, epaiss, rho):
        self.noms_params = noms_params
        self.Re = Re
        self.penalty = penalty
        self.epaiss = epaiss
        self.rho = rho

    def write_params(self, params):
        list_params = [(self.noms_params[i], params[i]) for i in range(len(params))]
        filename = 'C:/Users/TOUGERON/Documents/PRO/SMECA/ModeleTable_OCTAVE/Param_test.txt'
        with open(filename, 'w') as file:
            for param, value in list_params:
                file.write(f"{param} {value}\n")
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/EVOL/VAR/Param_evol.txt'
        with open(file_path, 'a') as file:
            file.write('-------\n')
            for param, value in list_params:
                file.write(f"{param} {value}\n")

    def read_output(self):
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table2_Files/RunCase_2/Result-Stage_1/res_const.txt'
        vm_values = []
        start_reading = False
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if "VMIS" in line:
                    start_reading = True
                if start_reading:
                    if '=====>' in line:
                        break
                    try:
                        vm = line.split("|")
                        if len(vm) <= 5:
                            break
                        else:
                            vm = vm[4]
                            vm = float(vm)
                            vm_values.append(vm)
                    except ValueError:
                        continue
        return vm_values

    def launch_calcul(self):
        try:
            result = subprocess.check_output('wsl -d smeca /opt/sm/bin/as_run /mnt/c/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table2_Files/RunCase_2/Result-Stage_1/export')
            return result
        except subprocess.CalledProcessError as e:
            print(f"Error during launch_calcul: {e}")
            return None

    def launch_mesh(self):
        try:
            result = subprocess.check_output('wsl -d smeca /opt/sm/bin/salome_meca shell python /mnt/c/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/scirpt1.py')
            return result
        except subprocess.CalledProcessError as e:
            print(f"Error during launch_mesh: {e}")
            return None

    def read_flamb(self):
        try:
            with open('C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table2_Files/RunCase_2/Result-Stage_1/resu_flamb', 'r') as file:
                lines = file.readlines()

            char_crit_values = []
            for line in lines:
                line = line.strip()
                if "=====>" in line:
                    break
                if line and not any(c.isalpha() for c in line.split()[1:]):
                    try:
                        char_crit = line.split()[1].replace('D', 'E')
                        char_crit = float(char_crit)
                        char_crit_values.append(char_crit)
                    except (IndexError, ValueError):
                        continue

            char_crit_array = np.array(char_crit_values)
            return char_crit_array
        except Exception as e:
            print(f"Error during read_flamb: {e}")
            return np.array([])

    def read_mass(self):
        try:
            file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table2_Files/RunCase_2/Result-Stage_1/masse.txt'
            with open(file_path, 'r') as file:
                lines = file.readlines()

            for line in lines:
                if line.startswith("Table"):
                    columns = line.split()
                    try:
                        mass = float(columns[2])
                        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/EVOL/OBJ/obj_val.txt'
                        with open(file_path, 'a') as file:
                            file.write('-------\n')
                            file.write(f"{mass}\n")
                        return mass
                    except (IndexError, ValueError) as e:
                        raise ValueError(f"Erreur lors de l'extraction de la masse : {e}")

            raise ValueError("La ligne contenant la masse n'a pas été trouvée.")
        except Exception as e:
            print(f"Error during read_mass: {e}")
            return None

    def perform_calculations(self, params, retries=3, delay=5):
        for attempt in range(retries):
            try:
                self.write_params(params)
                mesh_result = self.launch_mesh()
                if mesh_result is None:
                    raise Exception("Mesh generation failed.")
                calcul_result = self.launch_calcul()
                if calcul_result is None:
                    raise Exception("Calculation failed.")
                self.results_vm = self.read_output()
                self.results_flamb = self.read_flamb()
                self.mass = self.read_mass()
                if self.mass is None:
                    raise Exception("Mass reading failed.")
                return
            except Exception as e:
                print(f"Attempt {attempt + 1} failed with error: {e}")
                if attempt < retries - 1:
                    time.sleep(delay)
                else:
                    raise

    def objective(self, params):
        try:
            self.perform_calculations(params)
            return self.mass
        except Exception as e:
            print(f"Error in objective function: {e}")
            return float('inf')  # Return a high penalty value

    def constr_flamb(self, params):
        try:
            self.perform_calculations(params)
            coeff = np.abs(self.results_flamb)
            return min(coeff) - 1
        except Exception as e:
            print(f"Error in constraint function constr_flamb: {e}")
            return float('inf')  # Return a high penalty value

    def constr_vm(self, params):
        try:
            self.perform_calculations(params)
            coeff = np.abs(self.results_vm)
            return -max(coeff) + self.Re
        except Exception as e:
            print(f"Error in constraint function constr_vm: {e}")
            return float('inf')  # Return a high penalty value

# Créer une instance du modèle
noms_params = ['L1', 'L2', 'H', 'P']
Re = 235
penalty = 10e6
epaiss = 20
rho = 7800

model = MyModel2(noms_params, Re, penalty, epaiss, rho)

# Définir la fonction objectif pour OpenTURNS
def objective_function(params):
    return [model.objective(params)]

def constraint_flamb(params):
    return [model.constr_flamb(params)]

def constraint_vm(params):
    return [model.constr_vm(params)]

# Créer un problème d'optimisation
problem2 = ot.OptimizationProblem(ot.PythonFunction(len(noms_params), 1, objective_function))
problem2.setMinimization(True)
bounds = ot.Interval([500, 300, 400, 25], [3000, 2000, 1500, 100])
problem2.setBounds(bounds)
problem2.setInequalityConstraint(ot.PythonFunction(len(noms_params), 1, constraint_flamb))
problem2.setInequalityConstraint(ot.PythonFunction(len(noms_params), 1, constraint_vm))

# Générer la population initiale
uniform = ot.ComposedDistribution([ot.Uniform(500, 3000), ot.Uniform(300, 2000), ot.Uniform(400, 1500), ot.Uniform(25, 100)])
ot.RandomGenerator.SetSeed(0)
init_pop = uniform.getSample(10)

# Effacer les fichiers précédents
def clear_file(file_path):
    with open(file_path, 'w'):
        pass

file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/EVOL/OBJ/obj_val.txt'
clear_file(file_path)
file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/EVOL/VAR/Param_evol.txt'
clear_file(file_path)

# Exécuter l'algorithme d'optimisation
algo = ot.Pagmo(problem2, "gaco", init_pop)
algo.setMaximumIterationNumber(5)

algo.run()
result = algo.getResult()
optimal_point = result.getOptimalPoint()
optimal_value = result.getOptimalValue()

print("Meilleurs paramètres : ", optimal_point)
print("Valeur en ce point : ", optimal_value)


### Implémentation de la gestion des crashs par exploration du voisinage

Si un point cause des crashs, on retente le calcul en ce point 3 fois, puis on essaie des points dans le voisinage de ce point; si les points du voisinage ne fonctionnent pas, alors on renvoie plus l'infini pour exclure la zone des possibilités.


In [ ]:
import openturns as ot
import numpy as np
import subprocess
import time

class MyModel2:
    def __init__(self, noms_params, Re, penalty, epaiss, rho):
        self.noms_params = noms_params
        self.Re = Re
        self.penalty = penalty
        self.epaiss = epaiss
        self.rho = rho

    def write_params(self, params):
        list_params = [(self.noms_params[i], params[i]) for i in range(len(params))]
        filename = 'C:/Users/TOUGERON/Documents/PRO/SMECA/ModeleTable_OCTAVE/Param_test.txt'
        with open(filename, 'w') as file:
            for param, value in list_params:
                file.write(f"{param} {value}\n")
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/EVOL/VAR/Param_evol.txt'
        with open(file_path, 'a') as file:
            file.write('-------\n')
            for param, value in list_params:
                file.write(f"{param} {value}\n")

    def read_output(self):
        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table2_Files/RunCase_2/Result-Stage_1/res_const.txt'
        vm_values = []
        start_reading = False
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if "VMIS" in line:
                    start_reading = True
                if start_reading:
                    if '=====>' in line:
                        break
                    try:
                        vm = line.split("|")
                        if len(vm) <= 5:
                            break
                        else:
                            vm = vm[4]
                            vm = float(vm)
                            vm_values.append(vm)
                    except ValueError:
                        continue
        return vm_values

    def launch_calcul(self):
        try:
            result = subprocess.check_output('wsl -d smeca /opt/sm/bin/as_run /mnt/c/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table2_Files/RunCase_2/Result-Stage_1/export')
            return result
        except subprocess.CalledProcessError as e:
            print(f"Error during launch_calcul: {e}")
            return None

    def launch_mesh(self):
        try:
            result = subprocess.check_output('wsl -d smeca /opt/sm/bin/salome_meca shell python /mnt/c/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/scirpt1.py')
            return result
        except subprocess.CalledProcessError as e:
            print(f"Error during launch_mesh: {e}")
            return None

    def read_flamb(self):
        try:
            with open('C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table2_Files/RunCase_2/Result-Stage_1/resu_flamb', 'r') as file:
                lines = file.readlines()

            char_crit_values = []
            for line in lines:
                line = line.strip()
                if "=====>" in line:
                    break
                if line and not any(c.isalpha() for c in line.split()[1:]):
                    try:
                        char_crit = line.split()[1].replace('D', 'E')
                        char_crit = float(char_crit)
                        char_crit_values.append(char_crit)
                    except (IndexError, ValueError):
                        continue

            char_crit_array = np.array(char_crit_values)
            return char_crit_array
        except Exception as e:
            print(f"Error during read_flamb: {e}")
            return np.array([])

    def read_mass(self):
        try:
            file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/Table2_Files/RunCase_2/Result-Stage_1/masse.txt'
            with open(file_path, 'r') as file:
                lines = file.readlines()

            for line in lines:
                if line.startswith("Table"):
                    columns = line.split()
                    try:
                        mass = float(columns[2])
                        file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/EVOL/OBJ/obj_val.txt'
                        with open(file_path, 'a') as file:
                            file.write('-------\n')
                            file.write(f"{mass}\n")
                        return mass
                    except (IndexError, ValueError) as e:
                        raise ValueError(f"Erreur lors de l'extraction de la masse : {e}")

            raise ValueError("La ligne contenant la masse n'a pas été trouvée.")
        except Exception as e:
            print(f"Error during read_mass: {e}")
            return None

    def generate_neighbor(self, params, scale=0.1):
        perturbation = scale * np.random.randn(len(params))
        return np.array(params) + perturbation

    def perform_calculations(self, params, retries=3, delay=5, neighbor_retries=3):
        for attempt in range(retries):
            try:
                self.write_params(params)
                mesh_result = self.launch_mesh()
                if mesh_result is None:
                    raise Exception("Mesh generation failed.")
                calcul_result = self.launch_calcul()
                if calcul_result is None:
                    raise Exception("Calculation failed.")
                self.results_vm = self.read_output()
                self.results_flamb = self.read_flamb()
                self.mass = self.read_mass()
                if self.mass is None:
                    raise Exception("Mass reading failed.")
                return
            except Exception as e:
                print(f"Attempt {attempt + 1} failed with error: {e}")
                if attempt < retries - 1:
                    time.sleep(delay)
                else:
                    for neighbor_attempt in range(neighbor_retries):
                        try:
                            new_params = self.generate_neighbor(params)
                            print(f"Retrying with neighbor params: {new_params}")
                            self.write_params(new_params)
                            mesh_result = self.launch_mesh()
                            if mesh_result is None:
                                raise Exception("Mesh generation failed.")
                            calcul_result = self.launch_calcul()
                            if calcul_result is None:
                                raise Exception("Calculation failed.")
                            self.results_vm = self.read_output()
                            self.results_flamb = self.read_flamb()
                            self.mass = self.read_mass()
                            if self.mass is None:
                                raise Exception("Mass reading failed.")
                            return
                        except Exception as neighbor_e:
                            print(f"Neighbor attempt {neighbor_attempt + 1} failed with error: {neighbor_e}")
                            if neighbor_attempt < neighbor_retries - 1:
                                time.sleep(delay)
                            else:
                                raise

    def objective(self, params):
        try:
            self.perform_calculations(params)
            return self.mass
        except Exception as e:
            print(f"Error in objective function: {e}")
            return float('inf')  # Return a high penalty value

    def constr_flamb(self, params):
        try:
            self.perform_calculations(params)
            coeff = np.abs(self.results_flamb)
            return min(coeff) - 1
        except Exception as e:
            print(f"Error in constraint function constr_flamb: {e}")
            return float('inf')  # Return a high penalty value

    def constr_vm(self, params):
        try:
            self.perform_calculations(params)
            coeff = np.abs(self.results_vm)
            return -max(coeff) + self.Re
        except Exception as e:
            print(f"Error in constraint function constr_vm: {e}")
            return float('inf')  # Return a high penalty value

# Créer une instance du modèle
noms_params = ['L1', 'L2', 'H', 'P']
Re = 235
penalty = 10e6
epaiss = 20
rho = 7800

model = MyModel2(noms_params, Re, penalty, epaiss, rho)

# Définir la fonction objectif pour OpenTURNS
def objective_function(params):
    return [model.objective(params)]

def constraint_flamb(params):
    return [model.constr_flamb(params)]

def constraint_vm(params):
    return [model.constr_vm(params)]

# Créer un problème d'optimisation
problem2 = ot.OptimizationProblem(ot.PythonFunction(len(noms_params), 1, objective_function))
problem2.setMinimization(True)
bounds = ot.Interval([500, 300, 400, 25], [3000, 2000, 1500, 100])
problem2.setBounds(bounds)
problem2.setInequalityConstraint(ot.PythonFunction(len(noms_params), 1, constraint_flamb))
problem2.setInequalityConstraint(ot.PythonFunction(len(noms_params), 1, constraint_vm))

# Générer la population initiale
uniform = ot.ComposedDistribution([ot.Uniform(500, 3000), ot.Uniform(300, 2000), ot.Uniform(400, 1500), ot.Uniform(25, 100)])
ot.RandomGenerator.SetSeed(0)
init_pop = uniform.getSample(10)

# Effacer les fichiers précédents
def clear_file(file_path):
    with open(file_path, 'w'):
        pass

file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/EVOL/OBJ/obj_val.txt'
clear_file(file_path)
file_path = 'C:/Users/TOUGERON/Documents/PRO/SMECA/Modele_def/EVOL/VAR/Param_evol.txt'
clear_file(file_path)

# Exécuter l'algorithme d'optimisation
algo = ot.Pagmo(problem2, "gaco", init_pop)
algo.setMaximumIterationNumber(5)

algo.run()
result = algo.getResult()
optimal_point = result.getOptimalPoint()
optimal_value = result.getOptimalValue()

print("Meilleurs paramètres : ", optimal_point)
print("Valeur en ce point : ", optimal_value)
